In [47]:
import os
import pandas as pd 
import xml.etree.ElementTree as et 
import requests
import csv

In [4]:
# direct = os.fsencode('./BILLSTATUS-113-hr')
# df_list = []
# for file in os.listdir(direct):
#     filename = os.fsdecode(file)
#     if filename.endswith(".xml"): 
#         #print(os.path.join(direct, filename))

#         xtree = et.parse('./BILLSTATUS-113-hr/'+filename)
#         xroot = xtree.getroot() 

#         for node in xroot:
#             print(node)
#             count = 0 
#             for n in node.getchildren():
#                 #if n is not None:
#                 #print(n.text + ":" + str(count))
#                 print(node)
#                 for a in node.getchildren():
#                     print(a)
#                 count += 1
#             #bill_num = node.find("billNumber").text if node is not None else None
#             #bill_title = node.find("titles/item/title").text if node is not None else None
#             #policy = node.find("policyArea/name").text if node is not None else None
#             #spon = node.find("sponsors/item/fullName").text if node is not None else None
            
#             #rows.append({"bill": "HR "+ bill_num, "title": bill_title, 
#             #             "policy area": policy, "sponsor": spon})

#         #out_df = pd.DataFrame(rows, columns = df_cols)
#         #df_list.appenf(out_df)
#         continue
#     else:
#         continue

# xtree = et.parse('./BILLSTATUS-113-hr/BILLSTATUS-113hr2.xml')
# xroot = xtree.getroot()


# #todo: make a recursive function that goes through tree and returns our csv 'row'
# # returns a dictionary with keys as features and values as the feature values
# def getData(root):
#     features = dict()
#     getDataHelper(root, features)
    
#     return features

# def getDataHelper(node, featureMap):
#     if len(featureMap) < 12:
#         if node.tag == 'billNumber':
#             featureMap['billNumber'] = node.text
#         elif node.tag == 'amendments':
#             featureMap['amendments'] = node.text
#         elif node.tag == 'houseOfRepresentatives':
#             featureMap['actions'] = node.text
#         elif node.tag == 'sponsors':
#             for sponsor in node.getchildren()
        
#         for child in node.getchildren():
#             getDataHelper(child, featureMap)
        


In [121]:
def getDataFromDatabase(congress, billID):
    key = "K1ZpNtLlsKTmqMocP2D7rTWasPASfUtz7BicNNch"
    url = "https://api.propublica.org/congress/v1/" + str(congress) + "/bills/hr" + str(billID) +".json"
    result = requests.get(url, headers={"X-API-Key": key})
    json_result = result.json()
#     print(json_result)
    if json_result['status'] == 'ERROR':
        return None

    data = list()
    data.append(congress)
    data.append(billID)
    data.append(json_result['results'][0]['sponsor_party'])
    data.append(json_result['results'][0]['sponsor_state'])
    data.append(json_result['results'][0]['cosponsors'])
    try:
        data.append(json_result['results'][0]['cosponsors_by_party']['R'])
    except:
        data.append(0)
    try:
        data.append(json_result['results'][0]['cosponsors_by_party']['D'])
    except:
        data.append(0)
    try:
        data.append(json_result['results'][0]['cosponsors_by_party']['I'])
    except:
        data.append(0)
    data.append(json_result['results'][0]['primary_subject'])
    data.append(json_result['results'][0]['withdrawn_cosponsors'])
    data.append(len(json_result['results'][0]['committee_codes']))
    data.append(len(json_result['results'][0]['subcommittee_codes']))
    data.append(len(json_result['results'][0]['actions']))
    data.append(len(json_result['results'][0]['summary'].split(' ')))
    try:
        data += [1] if json_result['results'][0]['votes'][0]['result'] == "Passed" else [0]
    except:
        data.append(0)
    
    return [str(d) for d in data]

In [122]:
def writeDataToFile(filename, congress, hrStart, hrEnd):
    with open(filename, 'a') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in range(hrStart, hrEnd):
            data = getDataFromDatabase(congress, i)
            if(data != None):
                filewriter.writerow([str(i)] + data)
        
    csvfile.close()

In [131]:
# writeDataToFile('hr-bill-data.csv', '115', 0, 8000)